In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver #(conda install -c conda-forge selenium)

In [ ]:
r = requests.get('https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!/page/0/length/-1/sort_by/rank/sort_order/asc/cols/stats')
r.text

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')
soup.h1

In [ ]:
table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="datatable-1") 
rows = table.tbody.findAll(lambda tag: tag.name=='tr')

In [ ]:
body = table.find('thead')
body

In [ ]:
from selenium import webdriver

browser = webdriver.Chrome()  
browser.get('https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking#!/page/0/length/-1/sort_by/rank/sort_order/asc/cols/stats')  
html_source = browser.page_source  
browser.quit()

soup = BeautifulSoup(html_source, 'html.parser')  
table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="datatable-1") 
rows = table.tbody.findAll(lambda tag: tag.name=='tr')
print(rows)

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
SITE = "https://www.timeshighereducation.com"
URL = SITE+"/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json"

In [ ]:
N = 200
r = requests.get(URL)
all_data = r.json()
print(list(all_data.keys()))
data = all_data.get("data")[:N]
print(list(data[0].keys()))

name, rank, country and region, number of faculty members (international and total) and number of students (international and total)

In [ ]:
data[0]

In [ ]:
s = requests.Session()
reqs = [(req_id, s.prepare_request(requests.Request('GET', SITE+entry["url"]))) for req_id, entry in enumerate(data)]
print(len(reqs), "requests to be sent.")

resps = [(req_id, s.send(req)) for req_id, req in reqs]

done = [(req_id, resp.text) for req_id, resp in resps if resp.status_code == 200]
failed = [(req_id, resp) for req_id, resp in resps if resp.status_code != 200]

print("%d done, %d failed." % (len(done), len(failed)))

In [ ]:
uni = pd.DataFrame(data, columns=["name", "rank", "location", "stats_pc_intl_students", "stats_student_staff_ratio", "url"])
uni.rename(columns={"location":"country","stats_pc_intl_students":"pc_intl_students(%)", "stats_student_staff_ratio":"student_staff_ratio"},inplace=True)

In [ ]:
uni["rank"] = uni["rank"].str.extract('(\d+)', expand=False).astype(int)
uni["pc_intl_students(%)"]= uni["pc_intl_students(%)"].str.extract('(\d+)', expand=False).astype(int)

In [ ]:
uni